In [6]:
import ee
import geemap
import os
import geopandas as gpd
import numpy as np
import time
#start the timer
start = time.time()


In [7]:
#Mais de 50 petabytes de dados que são atualizados diariamente

#l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

#Mais de 900 Datasets de imagens de satélite e dados de sensoriamento remoto
#https://geemap.org/notebooks/46_local_rf_training/?h=model
#https://dynamicworld.app/


#stop the timer and calculate the total time in seconds
end = time.time()

print("Total time in seconds:", end - start)

Total time in seconds: 1.0132577419281006


In [8]:
### Stop this code for 10 seconds
stop = time.sleep(10)

#Calculating NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def calculate_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI')
    return image.addBands(ndwi)

def calculate_ndbi(image):
    ndbi = image.normalizedDifference(['B6', 'B5']).rename('NDBI')
    return image.addBands(ndbi)

#Selecionando diferentes recortes espaciais de acordo com o país
def FAO_Global_adm_select_municipalities(place):
    dataset = ee.FeatureCollection('FAO/GAUL/2015/level2')
    return dataset.filter(ee.Filter.eq('ADM2_NAME', place))

def FAO_Global_adm_select_states(place):
    dataset = ee.FeatureCollection('FAO/GAUL/2015/level1')
    return dataset.filter(ee.Filter.eq('ADM1_NAME', place))

def FAO_Global_adm_select_countries(place):
    dataset = ee.FeatureCollection('FAO/GAUL/2015/level0')
    return dataset.filter(ee.Filter.eq('ADM0_NAME', place))

#Carbon above and below ground
def carbon_above_and_below_ground(place):
    carbon_collection = ( 
    ee.ImageCollection('NASA/ORNL/biomass_carbon_density/v1')
    .filterBounds(place)
    .median()
    .clip(place)
    )
    return  carbon_collection


############ Dealing with clouds in sentinel-2 images #############################################
CLOUD_FILTER = 5
CLD_PRB_THRESH = 10
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 1
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)




########################################################################################################

def sentinel_2_images_median(place, initial_date, final_date, cloud_cover):
    """
    args:
        place: area de interesse
        initial_date: data inicial, formato 'YYYY-MM-DD', string
        final_date: data final, formato 'YYYY-MM-DD', string
        cloud_cover: nuvens, int
    returns: 
        imagem mediana do Sentinel-2 para a area de interesse, e data de download
    """

    s2_collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(initial_date, final_date) #selecionando data do download
    .filterBounds(place) #selecionando a area de interesse
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',
                          cloud_cover) #retirando nuvens (< 3%)
            )
    .median() #pegando mediana das imagens
    .clip(place) #clipando a area de interesse
    )
    return s2_collection

def leaf_area_index(place, initial_date, final_date):
    """
    Args:
        place: area de interesse
        initial_date: data inicial, formato 'YYYY-MM-DD', string
        final_date: data final, formato 'YYYY-MM-DD', string
    Returns:
        imagem mediana do LAI para a area de interesse
    
    """

    lai = (
    ee.ImageCollection('MODIS/061/MCD15A3H')
    .filterDate(initial_date, final_date) #selecionando data do download
    .filterBounds(place) #selecionando a area de interesse
    .first()
    .select('Fpar')
    .clip(place) #clipando a area de interesse
    )
    return lai


def download_set_per_image(output_path, var, filename, region, scale):
    """
    Args:
        output_path: caminho de destino do download
        var: variavel a ser baixada
        filename: nome do arquivo
        region: regiao de interesse
        scale: escala de resolucao
    Returns:
        string
    
    """
    output_path = os.path.expanduser(output_path)
    os.chdir(output_path)
    geemap.download_ee_image(var,
                        filename=filename,
                         region = region, 
                            crs = 'EPSG:3857',
                            scale = scale
    )
    return print('Download realizado com sucesso!')

def buffer_image(image, buffer):
    return image.clip(image.geometry().buffer(buffer))

In [9]:
Map = geemap.Map()
Map

AttributeError: module 'ee' has no attribute 'Reducer'

In [8]:
# #let the user define the area of interest
# aoi = Map.draw_last_feature
# aoi = aoi.geometry()
# Map.centerObject(aoi, 8)

# Run this cell and add your area of interest and dates (ex: "Cambuquira")
Municipality = "Cambuquira"

# Add the dates you wish to analyse, from start to finish, using the YYYY-MM-DD format (ex: '2022-05-01', '2022-10-31')
begin_analysis_date = "2022-05-01"
end_analysis_date = "2022-10-31"


In [9]:
## Run this cell to add the location of your drawing to the model
aoi = FAO_Global_adm_select_municipalities(Municipality)
aoi = aoi.geometry()
Map.centerObject(aoi, 11)
Map.addLayer(aoi, {}, 'aoi')

### Add the image with a date
s2_image = get_s2_sr_cld_col(aoi, begin_analysis_date, end_analysis_date)
image_final = (s2_image.map(add_cld_shdw_mask)
                                    .map(apply_cld_shdw_mask)
                                    .median()
                                    .clip(aoi)
                                    )
vis_params = {'min': 0, 'max': 3500, 'bands': ['B4', 'B3', 'B2']}

Map.addLayer(image_final, vis_params, "Primeira visualização")

In [7]:
### I will use DynamicWorld Dataset to train my model
filter = ee.Filter.And(ee.Filter.bounds(aoi), ee.Filter.date(begin_analysis_date, end_analysis_date))

dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(filter)
dynamic_world_image = ee.Image(dynamic_world.first())

CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]


VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dynamic_world_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)
# Get the most likely class probability.
dynamic_world = dynamic_world_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
dynamic_world = ee.Terrain.hillshade(dynamic_world.multiply(100)).divide(255)

Map.addLayer(dw_rgb, {}, 'DynamicWorld')


In [8]:
points = dynamic_world_image.sample(**{
    "region" : image_final.geometry(),
    "scale" : 10,
    "numPixels" : 5000,
    "seed" : 0,
    "geometries" : True,
}

)

Map.addLayer(points, {}, 'points')

In [11]:
### After adding the points, I will choose the bands for prediction
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']

label = "label"
# Overlay the points on the imagery to get training
training = image_final.select(bands).sampleRegions(**{
    'collection': points,
    'properties': ['label'],
    'scale': 10
})

#Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)


In [12]:
#Train a Random Forest classifier with default parameters.
trained = ee.Classifier.smileRandomForest(50).train(training, label, bands)

In [13]:
result = image_final.select(bands).classify(trained)
Map.addLayer(result.randomVisualizer(), {}, 'classfied', 
             #set opacity
                opacity = 0.5
             
             
             )
#set the pallette for the result using the same pallette as the dynamic world dataset


In [ ]:
### Defining Area of Interest as polygon ###
# path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/dnn_cacau/vault/past/aoi_dnn_cacau.shp'
path = gpd.read_file('/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/campo_7_agosto/shp/Restaura_limite_lotes_editando_v24agosto2023.shp')
#Drop all columns except geometry
path = path.drop(path.columns.difference(['geometry']), axis=1)
#Convert it to a FeatureCollection
var = geemap.geopandas_to_ee(path)

# var = geemap.shp_to_ee(path)


#Convert it to a FeatureCollection
geometry_var = var.geometry()
print(geometry_var.getInfo())
Map.addLayer(var, {})
Map.centerObject(var, 7)
Map

### Running cloud mask with the limits of Area of Interest ###
s2_sr_cld_col_eval = get_s2_sr_cld_col(geometry_var, '2023-05-01', '2023-10-31')

s2_sr_median = (s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                                    .map(apply_cld_shdw_mask)
                                    .median()
                                    .clip(geometry_var)
                                    )
Map.addLayer(s2_sr_median,
              {'bands': ['B4', 'B3', 'B2'],
             'min': 0, 'max': 3500,'gamma': 1.0}, 'S2 SR masked')

### Selecting the bands of interest ###
bands_10_meters_res =  s2_sr_median.select(['B4','B3','B2'])
bands_20_meters_res = s2_sr_median.select(['B4','B3','B2', 'B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'])
bands_20_meters_res_t = s2_sr_median.select(['B11', 'B12'])

### Exporting the image ###
projection="EPSG:3857" # 31982, SIRGAS 22s
out_dir = os.path.expanduser('~/Downloads')
os.chdir(out_dir)

# geemap.download_ee_image(bands_10_meters_res,
#                           filename='sentinel_2_2023_aoi_10m_3_channels_int16.tif',
#                           region=geometry_var,       
#                           crs = 'EPSG:4326',
#                             scale=10,
#                               overwrite=True, 
#                               dtype='int16',
#                                 max_tile_size=16, 
#                               )
  

In [ ]:
Map

In [ ]:
//

In [ ]:
### Now I will download the NDVI, NDWI and NDBI ###
s2_sr_median_ndvi = calculate_ndvi(s2_sr_median)
s2_sr_median_ndwi = calculate_ndwi(s2_sr_median)

#Add it to the map
Map.addLayer(s2_sr_median_ndvi, {'bands': ['NDVI'], 'min': 0, 'max': 1}, 'NDVI')
Map.addLayer(s2_sr_median_ndwi, {'bands': ['NDWI'], 'min': 0, 'max': 1}, 'NDWI')

geemap.download_ee_image(s2_sr_median_ndvi,
                            filename='sentinel_2_2023_aoi_ndvi.tif',
                            region=geometry_var,       
                            crs = 'EPSG:4326',
                                scale=30,
                                overwrite=True, 
                                dtype='int32',
                                    max_tile_size=16, 
                                )

In [ ]:
print(type(s2_sr_median_ndvi))

In [ ]:
////

In [ ]:
### Defining Area of Interest as polygon ###
path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/dnn_cacau/vault/past/aoi_dnn_cacau.shp'

var = geemap.shp_to_ee(path)
#Convert it to a FeatureCollection
geometry_var = var.geometry()
print(geometry_var.getInfo())
Map.addLayer(var, {})
Map.centerObject(var, 7)
Map

### Running cloud mask with the limits of Area of Interest ###
s2_sr_cld_col_eval = get_s2_sr_cld_col(geometry_var, '2021-05-01', '2021-10-31')

s2_sr_median = (s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                                    .map(apply_cld_shdw_mask)
                                    .median()
                                    .clip(geometry_var)


                                    )
Map.addLayer(s2_sr_median,
              {'bands': ['B4', 'B3', 'B2'],
             'min': 0, 'max': 3500,'gamma': 1.0}, 'S2 SR masked')
Map

### Selecting the bands of interest ###
bands_10_meters_res =  s2_sr_median.select(['B4','B3','B2'])
bands_20_meters_res = s2_sr_median.select(['B4','B3','B2', 'B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'])
bands_20_meters_res_t = s2_sr_median.select(['B11', 'B12'])

### Exporting the image ###
projection="EPSG:3857" # 31982, SIRGAS 22s
out_dir = os.path.expanduser('~/Downloads')
os.chdir(out_dir)

geemap.download_ee_image(bands_10_meters_res,
                          filename='sentinel_2_2021_aoi_10m_3_channels_int16.tif',
                          region=geometry_var,       
                          crs = 'EPSG:4326',
                            scale=10,
                              overwrite=True, 
                              dtype='int16',
                                max_tile_size=10, 
                              )
  

In [ ]:
### Defining Area of Interest as polygon ###
path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/dnn_cacau/vault/past/aoi_dnn_cacau.shp'

var = geemap.shp_to_ee(path)
#Convert it to a FeatureCollection
geometry_var = var.geometry()
print(geometry_var.getInfo())
Map.addLayer(var, {})
Map.centerObject(var, 7)
Map

### Running cloud mask with the limits of Area of Interest ###
s2_sr_cld_col_eval = get_s2_sr_cld_col(geometry_var, '2020-05-01', '2020-10-31')

s2_sr_median = (s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                                    .map(apply_cld_shdw_mask)
                                    .median()
                                    .clip(geometry_var)


                                    )
Map.addLayer(s2_sr_median,
              {'bands': ['B4', 'B3', 'B2'],
             'min': 0, 'max': 3500,'gamma': 1.0}, 'S2 SR masked')
Map

### Selecting the bands of interest ###
bands_10_meters_res =  s2_sr_median.select(['B4','B3','B2'])
bands_20_meters_res = s2_sr_median.select(['B4','B3','B2', 'B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'])
bands_20_meters_res_t = s2_sr_median.select(['B11', 'B12'])

### Exporting the image ###
projection="EPSG:3857" # 31982, SIRGAS 22s
out_dir = os.path.expanduser('~/Downloads')
os.chdir(out_dir)

geemap.download_ee_image(bands_10_meters_res,
                          filename='sentinel_2_2020_aoi_10m_3_channels_int16.tif',
                          region=geometry_var,       
                          crs = 'EPSG:4326',
                            scale=10,
                              overwrite=True, 
                              dtype='int16',
                                max_tile_size=10, 
                              )
  

In [ ]:
### Defining Area of Interest as polygon ###
path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/dnn_cacau/vault/past/aoi_dnn_cacau.shp'

var = geemap.shp_to_ee(path)
#Convert it to a FeatureCollection
geometry_var = var.geometry()
print(geometry_var.getInfo())
Map.addLayer(var, {})
Map.centerObject(var, 7)
Map

### Running cloud mask with the limits of Area of Interest ###
s2_sr_cld_col_eval = get_s2_sr_cld_col(geometry_var, '2019-05-01', '2019-10-31')

s2_sr_median = (s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                                    .map(apply_cld_shdw_mask)
                                    .median()
                                    .clip(geometry_var)


                                    )
Map.addLayer(s2_sr_median,
              {'bands': ['B4', 'B3', 'B2'],
             'min': 0, 'max': 3500,'gamma': 1.0}, 'S2 SR masked')
Map

### Selecting the bands of interest ###
bands_10_meters_res =  s2_sr_median.select(['B4','B3','B2'])
bands_20_meters_res = s2_sr_median.select(['B4','B3','B2', 'B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'])
bands_20_meters_res_t = s2_sr_median.select(['B11', 'B12'])

### Exporting the image ###
projection="EPSG:3857" # 31982, SIRGAS 22s
out_dir = os.path.expanduser('~/Downloads')
os.chdir(out_dir)

geemap.download_ee_image(bands_10_meters_res,
                          filename='sentinel_2_2019_aoi_10m_3_channels_int16.tif',
                          region=geometry_var,       
                          crs = 'EPSG:4326',
                            scale=10,
                              overwrite=True, 
                              dtype='int16',
                                max_tile_size=10, 
                              )
  

In [ ]:
### Defining Area of Interest as polygon ###
path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/dnn_cacau/vault/past/aoi_dnn_cacau.shp'

var = geemap.shp_to_ee(path)
#Convert it to a FeatureCollection
geometry_var = var.geometry()
print(geometry_var.getInfo())
Map.addLayer(var, {})
Map.centerObject(var, 7)
Map

### Running cloud mask with the limits of Area of Interest ###
s2_sr_cld_col_eval = get_s2_sr_cld_col(geometry_var, '2019-05-01', '2019-10-31')

s2_sr_median = (s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                                    .map(apply_cld_shdw_mask)
                                    .median()
                                    .clip(geometry_var)


                                    )
Map.addLayer(s2_sr_median,
              {'bands': ['B4', 'B3', 'B2'],
             'min': 0, 'max': 3500,'gamma': 1.0}, 'S2 SR masked')
Map

### Selecting the bands of interest ###
bands_10_meters_res =  s2_sr_median.select(['B4','B3','B2'])
bands_20_meters_res = s2_sr_median.select(['B4','B3','B2', 'B5','B6','B7', 'B8', 'B8A', 'B11', 'B12'])
bands_20_meters_res_t = s2_sr_median.select(['B11', 'B12'])

### Exporting the image ###
projection="EPSG:3857" # 31982, SIRGAS 22s
out_dir = os.path.expanduser('~/Downloads')
os.chdir(out_dir)


# zip the files before downloading
geemap.download_ee_image(bands_10_meters_res,
                          filename='sentinel_2_2018_aoi_10m_3_channels_int16.tif',
                          region=geometry_var,       
                          crs = 'EPSG:4326',
                            scale=10,
                              overwrite=True, 
                              dtype='int16',
                                max_tile_size=10, 
                              )
  

In [ ]:
import pygame
#if code is runs, then play music
pygame.init()
pygame.mixer.music.load('/home/luisthethormes/01_sol/geo-automate-and-interconnect-systems/Automate_processes/Outro/Music/06_Secret_Jingle_Zelda.mp3')
pygame.mixer.music.set_volume(1)
pygame.mixer.music.play()


In [ ]:
///

In [ ]:
#Setting ROI same as RestauraAmazonia
#Quando a entrada for um shapefile, esta parte do código transforma em um objeto do tipo ee.Geometry
#E depois transforma em um objeto geométrico do tipo ee.FeatureCollection
# path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/modelagem_desmate/grades_validacao_merge.shp'
# roi = geemap.shp_to_ee(path)
# print(type(roi))
# geometry_roi = roi.geometry()
# print(geometry_roi.getInfo())
# Map.addLayer(roi, {}, 'ROI')

In [ ]:
# path = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/sdu_mais_geo/bd_01/limite_propriedades/Limite_Propriedades_amostradas_29_03_2023.shp'
# limite_prop = geemap.shp_to_ee(path)
# print(type(limite_prop))
# geometry_limite_prop = roi.geometry()
# print(geometry_roi.getInfo())
# #Add the ROI to the map 
# Map.addLayer(roi, {}, 'ROI')

In [ ]:
# path  = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/unet_cacau/shp/aoi_cacau_pleno_sol_treinamento_pre_label.shp'
# limite_validacao = geemap.shp_to_ee(path)
# print(type(limite_validacao))
# geometry_limite_validacao = limite_validacao.geometry()
# print(geometry_limite_validacao.getInfo())
# #Add the ROI to the map
# Map.addLayer(limite_validacao, {}, 'Limite_validacao')

Download de uma imagem composta

In [ ]:
#Criação das máscara de nuvem usando Sentinel-2 QA60 band.
def mask2clouds(image):
    qa = image.select('QA60')
    #Bits 10 e 11 são nuvens e cirrus, respectivamente.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    #Ambos devem ser definidos em zero, indicando condições claras.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)


cloud_cover =  6 #porcentagem de nuvens
## Download da imagem mediana do Sentinel-2 para a area de interesse
cloudBitMask = 1 << 10
cirrusBitMask = 1 << 11
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-08-01', '2021-10-31') #selecionando data do download
    .filterBounds(var) #selecionando a area de interesse
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE'
                    ,'less_than',
                      cloud_cover) #retirando nuvens (< 3%)
    .map(mask2clouds)
    .median() #pegando mediana das imagens
    .clip(var)
      #clipando a area de interesse
    )
## To collection apply the mask2clouds function using python
collection = ee.ImageCollection.fromImages(collection)
collection = collection.map(mask2clouds)

## To Image again
image = ee.Image(collection)


In [ ]:
## Band combinations
rgb_nir  = collection.select(['B2', 'B3', 'B4', 'B8']) 
b11_b08_B12 =  collection.select(['B8', 'B11', 'B12']) 
all_bands = collection.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B8', 'B9', 'B11', 'B12'])
bands_10_meters_res =  collection.select(['B4','B3','B2'])

# print(collection.getInfo()) #printando as informacoes da imagem

In [ ]:
# Map.addLayer(rgb_nir, {'min': 0, 'max': 3000}, 'Sentinel-2,  RGB + NIR') #adicionando a imagem ao mapa
# Map.addLayer(b11_b08_B12, {'min': 0, 'max': 3000}, 'Sentinel-2,  B11 + B08 + B12') #adicionando a imagem ao mapa


In [ ]:
Map.addLayer(image,
              {'bands': ['B4', 'B3', 'B2'],
                'min': 0, 'max': 3000})
                  

# #Plot image
# Map.addLayer(ndvi,
#               {'bands': ['NDVI'],
#                'min': 0, 'max': 1,
#                'palette': ['red', 'yellow', 'green']},
#                'NDVI')

In [ ]:
# projection="EPSG:3857"
# out_dir = os.path.expanduser('~/Downloads')
# os.chdir(out_dir)

# geemap.download_ee_image(bands_10_meters_res,
#                         filename='sentinel_2_2021_nr_cacau_detection_bands_10m_res.tif',
#                          region=geometry_var,       
#                          crs = 'EPSG:3857',
#                           scale=10,
#                             overwrite=True
#                             )
# geemap.download_ee_image(ndvi,
#                         filename='sentinel_2_2021_nr_cacau_detection_ndvi.tif',
#                           region=geometry_var,
#                           crs = 'EPSG:3857',
#                           scale=10,
#                             overwrite=True
#                             )


Download de imagem por tile (caso imagem seja muito grande)

In [ ]:
# ###Setting the output directory
# projection="EPSG:3857"
# out_dir = os.path.expanduser('~/Downloads')
# os.chdir(out_dir)

### Creating fishnet grid
region = var
fishnet =  geemap.fishnet(region, rows=8, cols=8,  delta = 1)


Map.addLayer(fishnet, {}, "fishnet!")
Map
geemap.download_ee_image_tiles(bands_10_meters_res,
                               features = fishnet,
                                 out_dir=out_dir,
                                    scale=30,
                                    prefix='sentinel_2_image_int_16_',
                                    crs='EPSG:3857', 
                                    overwrite=True, 
                                    dtype='float32', 
                                    max_tile_dim=10000, max_tile_size=5000)
                                 

In [ ]:
import pygame
#if code is runs, then play music
pygame.init()
pygame.mixer.music.load('/home/luisthethormes/01_sol/geo-automate-and-interconnect-systems/Automate_processes/Outro/Music/06_Secret_Jingle_Zelda.mp3')
pygame.mixer.music.set_volume(1)
pygame.mixer.music.play()

In [ ]:
#Merging all tiles in one image using gdal_merge.py
import os
import glob
import subprocess 
import numpy as np


#Setting the output directory
out_dir = os.path.expanduser('~/Downloads')
os.chdir(out_dir)
list_tif = glob.glob('sentinel_2_image_int_16_*.tif')
list_tif.sort()
print(list_tif)        

In [ ]:
///

In [ ]:
import subprocess
full_apath_image = '/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/unet_cacau/data/sentinel_2/images_treino/clip_image_treino_2022.tif'
output_path_image =  full_apath_image.replace('.tif', '.jpg')

#Using subprocess, convert .tif to .png
def create_tif_RGB(tif_file, output):
    """
    Args: 
        tif_file: path to the tif file
        output: path to the output file
    Returns:
        output: path to the output file, and RGB tif file
    """
    #Select the RGB bands
    subprocess.call(['gdal_translate',
                        '-of',
                         'GTIFF',
                            '-b', #Red band
                            '1',
                            '-b', #Green band
                            '2',
                            '-b', #Blue band
                            '3',
                            tif_file,
                            output])
    return output


def tif_to_8_bits(tif_file, output):
    #Select the RGB bands
    subprocess.call(["gdal_translate",
                      "-ot",
                        "Byte",
                          "-scale",
                            tif_file,
                            output])
    return output


In [ ]:
#First, generate the RGB tif file and then convert it to 8 bits
tif_to_8_bits(full_apath_image, output_path_image)

In [ ]:
#Abrindo arquivos com as máscaras (banco de dados para treino)
from skimage import io
import numpy as np
from matplotlib import pyplot as plt

teste_mask_np =  np.load('/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/unet_cacau/data/sentinel_2/images_treino/npy/mask_aoi_treino.npy')
teste_mask_png = io.imread('/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/unet_cacau/data/sentinel_2/images_treino/png/mask_aoi_treino.png')

#binarizando a máscara
teste_mask = np.where(teste_mask_png > 0, 1, teste_mask_png)
plt.imshow(teste_mask, cmap='gray')

#Checando o shape da máscara, tamanho da imagem e número de classes
print(teste_mask.shape)

#print number of classes
print(np.unique(teste_mask))

#Export numpy array to tif
from PIL import Image
import numpy as np
import os

image = Image.fromarray(teste_mask)
image.save('/media/luisthethormes/Seagate Expansion Drive/Louis_work/02_Solidaridad_network/unet_cacau/data/sentinel_2/images_treino/tif/mask_aoi_treino.tif')



In [ ]:
# ###Setting the output directory
# projection="EPSG:3857"
# out_dir = os.path.expanduser('~/Downloads')
# os.chdir(out_dir)

In [ ]:
#open .tif file as array, use GDAL 
import numpy as np
import os
import rasterio as rio
import subprocess
path_root = 'home/luisthethormes/Downloads'
path_tif = '/home/luisthethormes/Downloads/sentinel_2_2021_aoi_for_cacau_detection_bands_10m_res.tif'
output = path_tif.replace('.tif', '_float_32.tif')
def raster_float_64_to_float_32(path_tif, output):
    """
    Args:
        path_tif: path to the tif file
    Returns:
        output: path to the output file, and RGB tif file
    """
    #Select the RGB bands
    subprocess.call(['gdal_translate',
                        '-ot',
                         'Float32',
                            path_tif,
                            output])
    return path_tif
raster_float_64_to_float_32(path_tif, output)

In [ ]:
import os
import subprocess
path_shp = '/home/luisthethormes/Downloads/Export_Output_2.shp'
output = path_shp.replace('.shp', '_rasterized_.tif')
def shapefile_to_raster(path_shp, output):
    """
    Args:
        path_shp: path to the shapefile
    Returns:
        output: path to the output file, and RGB tif file
    """
    #Select the RGB bands
    subprocess.call(['gdal_rasterize', '-a', 'fid_1', '-a_nodata', '0', '-tr', '10', '10', '-ot', 'Int16', '-of', 'GTiff', path_shp, output])
    return path_shp
shapefile_to_raster(path_shp, output)

In [ ]:
help('gdal_rasterize')
subprocess.call(['gdal_rasterize', '-a', 'fid_1', '-a_nodata', '0', '-tr', '10', '10', '-ot', 'Int16', '-of', 'GTiff', path_shp, output])